# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_file = "input/weather_data.csv"
weather_data_file
weather_df = pd.read_csv(weather_data_file)
weather_df

,City,Lat,Lng,Max Temp in F,Humidity,Cloudiness,Wind Speed,Country,Date
0,port alfred,-33.5906,26.8910,66.002,89,38,3.73,ZA,1612487409
1,hithadhoo,-0.6000,73.0833,82.076,77,100,4.97,MV,1612487597
2,atuona,-9.8000,-139.0333,78.656,71,4,4.54,PF,1612487597
3,bhuj,23.2667,69.6667,58.568,48,0,3.57,IN,1612487598
4,ushuaia,-54.8000,-68.3000,46.400,81,20,15.43,AR,1612487346
...,...,...,...,...,...,...,...,...,...
603,krasnoselkup,65.7000,82.4667,-23.620,86,36,2.98,RU,1612487945
604,dukat,62.5500,155.5500,-20.542,84,100,1.97,RU,1612487946
605,chapais,49.7834,-74.8492,15.800,86,1,1.19,CA,1612487946
606,ahipara,-35.1667,173.1667,70.466,63,100,2.22,NZ,1612487947


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)



In [82]:
fig = gmaps.figure(zoom_level=2, center=(25,5),layout={'width':'950px', 'height':'600px'})

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=weather_df["Humidity"].max(),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='600px', width='950px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
vacation=weather_df.loc[weather_df["Cloudiness"]<50]
vacation=vacation.loc[vacation["Humidity"]<75]
vacation=vacation.loc[vacation["Max Temp in F"]>0]
vacation=vacation.loc[vacation["Max Temp in F"]<32]
vacation

,City,Lat,Lng,Max Temp in F,Humidity,Cloudiness,Wind Speed,Country,Date
43,birjand,32.8663,59.2211,28.400,22,0,3.09,IR,1612487653
46,albany,42.6001,-73.9662,28.400,69,20,1.41,US,1612487654
76,chengde,40.9725,117.9361,28.202,74,0,1.12,CN,1612487604
84,petropavlovsk-kamchatskiy,53.0452,158.6483,12.200,48,20,1.00,RU,1612487611
141,jiutai,44.1525,125.8328,19.400,58,0,6.00,CN,1612487701
164,ilulissat,69.2167,-51.1000,1.400,35,20,4.12,GL,1612487713
185,linxia,35.6003,103.2064,26.042,70,0,0.35,CN,1612487725
206,mitchell,43.7094,-98.0298,19.400,63,1,4.12,US,1612487737
222,husavik,66.0449,-17.3389,26.600,63,0,4.12,IS,1612487747
279,varhaug,58.6147,5.6457,17.006,62,0,3.09,NO,1612487776


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [76]:
hotel_df=vacation
hotel_df["Hotel Name"]=""
hotel_df_us=hotel_df.loc[hotel_df["Country"]== "US"]
hotel_df_us

,City,Lat,Lng,Max Temp in F,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
46,albany,42.6001,-73.9662,28.4,69,20,1.41,US,1612487654,
206,mitchell,43.7094,-98.0298,19.4,63,1,4.12,US,1612487737,
525,cody,44.5263,-109.0565,30.2,40,40,5.66,US,1612487903,


In [88]:
hotel_df

,City,Lat,Lng,Max Temp in F,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
43,birjand,32.8663,59.2211,28.400,22,0,3.09,IR,1612487653,Birjand Tourist Hotel
46,albany,42.6001,-73.9662,28.400,69,20,1.41,US,1612487654,Tru by Hilton Albany Crossgates Mall
76,chengde,40.9725,117.9361,28.202,74,0,1.12,CN,1612487604,Chengde Hotel
84,petropavlovsk-kamchatskiy,53.0452,158.6483,12.200,48,20,1.00,RU,1612487611,ATLAS
141,jiutai,44.1525,125.8328,19.400,58,0,6.00,CN,1612487701,Bali Laguna Hotel
164,ilulissat,69.2167,-51.1000,1.400,35,20,4.12,GL,1612487713,Best Western Plus Hotel Ilulissat
185,linxia,35.6003,103.2064,26.042,70,0,0.35,CN,1612487725,Hehai Hotel
206,mitchell,43.7094,-98.0298,19.400,63,1,4.12,US,1612487737,Coachlight Motel
222,husavik,66.0449,-17.3389,26.600,63,0,4.12,IS,1612487747,Post-Plaza Guesthouse
279,varhaug,58.6147,5.6457,17.006,62,0,3.09,NO,1612487776,Holmavatn Ungdoms og Misjonssenter


In [87]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"rankby": "distance",
        "type": "lodging",
        "keyword":"hotel",
        "key": gkey,
        }
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    city_name = row['City']
    # add location to params dict
    params['location'] = f"{hotel_df.at[index,'Lat']}, {hotel_df.at[index,'Lng']}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    #try:
    print(f"Closest hotel to {city_name} is {results[0]['name']}.")
    hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
                
    #except (KeyError, IndexError):
#print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 43: birjand.
Closest hotel to birjand is Birjand Tourist Hotel.
------------
Retrieving Results for Index 46: albany.
Closest hotel to albany is Tru by Hilton Albany Crossgates Mall.
------------
Retrieving Results for Index 76: chengde.
Closest hotel to chengde is Chengde Hotel.
------------
Retrieving Results for Index 84: petropavlovsk-kamchatskiy.
Closest hotel to petropavlovsk-kamchatskiy is ATLAS.
------------
Retrieving Results for Index 141: jiutai.
Closest hotel to jiutai is Bali Laguna Hotel.
------------
Retrieving Results for Index 164: ilulissat.
Closest hotel to ilulissat is Best Western Plus Hotel Ilulissat.
------------
Retrieving Results for Index 185: linxia.
Closest hotel to linxia is Hehai Hotel.
------------
Retrieving Results for Index 206: mitchell.
Closest hotel to mitchell is Coachlight Motel.
------------
Retrieving Results for Index 222: husavik.
Closest hotel to husavik is Post-Plaza Guesthouse.
------------
Retrieving Results fo

In [89]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [92]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='600px', width='950px'))